# Algoritmo de Classificação com XGBoost
### Conceitos de Ensemble/Boosting
<hr>

## Importando Pacotes

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score

import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')

## Carregando Dados

In [49]:
df_iris = sns.load_dataset('iris')
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [50]:
print(f'Quantidade de Linhas: {df_iris.shape[0]}\nQuantidade de Colunas: {df_iris.shape[1]}')

Quantidade de Linhas: 150
Quantidade de Colunas: 5


## Pré-Processamento dos Dados

In [51]:
df_iris['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [52]:
df_iris['species'] = df_iris['species'].map({'setosa':0, 'versicolor':1, 'virginica':2 })
print(f'Contagem:\n{df_iris.species.value_counts()}')

Contagem:
0    50
1    50
2    50
Name: species, dtype: int64


## Separando as Bases

In [53]:
x_iris = np.array(df_iris.drop(['species'], axis=1))
y_iris = np.array(df_iris['species'])

In [54]:
x_iris_train, x_iris_test, y_iris_train, y_iris_test = train_test_split(x_iris, y_iris, random_state=42)

In [55]:
print(f'Quantidade de dados para treino: {x_iris_train.shape[0]}\nQuantidade de dados para teste: {x_iris_test.shape[0]}\nTotal: {x_iris_train.shape[0] + x_iris_test.shape[0]}')
print(f'\nPorcentagem de dados para teste: {len(x_iris_test) / len(x_iris) *100:.2f}%')

Quantidade de dados para treino: 112
Quantidade de dados para teste: 38
Total: 150

Porcentagem de dados para teste: 25.33%


## Avaliando Modelos

In [56]:
classificador_xgb = xgb.XGBClassifier(verbosity = 0)
resultado_cv = cross_val_score(classificador_xgb, x_iris_train, y_iris_train)
print(f'\nResultado da avaliação: {resultado_cv}\nAcuracia: {resultado_cv.mean()*100:.2f}')


Resultado da avaliação: [0.95652174 0.95652174 0.86363636 0.95454545 0.95454545]
Acuracia: 93.72


In [57]:
classificador_xgb_tunado = xgb.XGBClassifier(max_depth=2, verbosity = 0)
resultado_cv_xgb = cross_val_score(classificador_xgb_tunado, x_iris_train, y_iris_train)
print(f'\nResultado da avaliação: {resultado_cv_xgb}\nAcuracia: {resultado_cv_xgb.mean()*100:.2f}')


Resultado da avaliação: [0.95652174 0.91304348 0.86363636 0.95454545 0.95454545]
Acuracia: 92.85


In [58]:
classificador_xgb_dart = xgb.XGBClassifier(booster='dart', verbosity = 0)
resultado_cv_dart = cross_val_score(classificador_xgb_dart, x_iris_train, y_iris_train)
print(f'\nResultado da avaliação: {resultado_cv_dart}\nAcuracia: {resultado_cv_dart.mean()*100:.2f}')


Resultado da avaliação: [0.95652174 0.95652174 0.86363636 0.95454545 0.95454545]
Acuracia: 93.72


In [59]:
classificador_xgb_gblinear = xgb.XGBClassifier(booster='gblinear', verbosity = 0)
resultado_cv_gblinear = cross_val_score(classificador_xgb_gblinear, x_iris_train, y_iris_train)
print(f'\nResultado da avaliação: {resultado_cv_gblinear}\nAcuracia: {resultado_cv_gblinear.mean()*100:.2f}')


Resultado da avaliação: [0.95652174 0.95652174 0.81818182 1.         1.        ]
Acuracia: 94.62


## Modelo Campeão

In [60]:
classificador_campeao = classificador_xgb_gblinear
print(f'Objetivo: {classificador_campeao.objective}\nTipo: {type(classificador_campeao)}')

Objetivo: binary:logistic
Tipo: <class 'xgboost.sklearn.XGBClassifier'>


## Treino

In [61]:
classificador_campeao.fit(x_iris_train, y_iris_train)

XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=-1,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.5, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=4, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, random_state=0,
              reg_alpha=0, reg_lambda=0, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=1, verbosity=0)

## Teste

In [62]:
predicoes_iris = classificador_campeao.predict(x_iris_test)

## Avaliação do Modelo

In [63]:
"""
acertos = (predicoes_iris == y_iris_test).sum()
total = len(y_iris_test)
acuracia = (acertos / total) * 100
acuracia
"""

'\nacertos = (predicoes_iris == y_iris_test).sum()\ntotal = len(y_iris_test)\nacuracia = (acertos / total) * 100\nacuracia\n'

In [64]:
Report = classification_report(y_iris_test, predicoes_iris)
print(f'{Report}\nAcurácia: {round(accuracy_score(y_iris_test, predicoes_iris)*100,2)}')

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.91      0.95        11
           2       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

Acurácia: 97.37


## Matriz de Confusão

In [65]:
print(pd.crosstab(y_iris_test, predicoes_iris, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1   2  All
Real                    
0        15   0   0   15
1         0  10   1   11
2         0   0  12   12
All      15  10  13   38


## Salvando Dados

In [74]:
df  =  pd.DataFrame(x_iris_test, columns=['sepal_length','sepal_width','petal_length','petal_width'])
df['SpeciesPreditas'] = predicoes_iris

In [75]:
df

,sepal_length,sepal_width,petal_length,petal_width,SpeciesPreditas
0,6.1,2.8,4.7,1.2,1
1,5.7,3.8,1.7,0.3,0
2,7.7,2.6,6.9,2.3,2
3,6.0,2.9,4.5,1.5,1
4,6.8,2.8,4.8,1.4,1
5,5.4,3.4,1.5,0.4,0
6,5.6,2.9,3.6,1.3,1
7,6.9,3.1,5.1,2.3,2
8,6.2,2.2,4.5,1.5,2
9,5.8,2.7,3.9,1.2,1


In [77]:
df.to_csv('../output/01_iris_classification_Model.csv', sep=';', index=False)

## Salvando o Modelo

In [79]:
joblib.dump(classificador_campeao, '../model/01_iris_classification_Model.pkl')

['../model/01_iris_classification_Model.pkl']

## Testando o Modelo

In [80]:
loaded_model = joblib.load('../model/01_iris_classification_Model.pkl')
new_data = sns.load_dataset('iris')
new_data.drop('species', inplace=True, axis=1)
new_data.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [81]:
result = loaded_model.predict(new_data)
df_new, df_new['SpeciesPreditas'] = new_data, result
df_new

,sepal_length,sepal_width,petal_length,petal_width,SpeciesPreditas
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
